In [12]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
#from LIM import utils as ut
import utils as ut
from LIM import LIM_class
#plt.style.use("./plotting.mplstyle")
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
data = xr.open_dataset("C:/Users/felix/PycharmProjects/ML_Climate_Science_Research_Project/LIM/data/ts_Amon_CESM2_piControl_r1i1p1f1.nc")["ts"]
#data = xr.open_dataset("./data/zos_Amon_CESM2_piControl_r1i1p1f1.nc")["zos"]
#data_old = xr.open_dataset("./data/ssta_1950_2021.nc")["ssta"]
mask = xr.open_dataset("C:/Users/felix/PycharmProjects/ML_Climate_Science_Research_Project/LIM/data/sftlf_fx_CESM2_historical_r1i1p1f1.nc")["sftlf"]

#14400 orginial size
data = data[:10000, :, :]

data = ut.apply_mask(mask, data)
#print("Data : {} + shape {}".format(data, data.shape))

data_anomalies = ut.calculate_monthly_anomalies(data)
#print("Month mean : {} + shape : {}".format(data_anomalies, data_anomalies.shape))

data_cropped =ut.crop_xarray(data_anomalies)
#print("Data cropped : {} + shape : {}".format(data_cropped, data_cropped.shape))


pca_10 = ut.SpatioTemporalPCA(data_cropped, n_components=20)
#pca_10 = ut.SpatioTemporalPCA(data_anomalies, n_components=20)
eof_10 = pca_10.eofs()
pc_10 = pca_10.principal_components()

# Create training and test data

data = pca_10.principal_components()
index_train = int(0.8 * len(data["time"]))
data_train = data[:, :index_train]
data_test = data[:, index_train:]

eps = 0.01

# Get input data
x_input = data_train.isel(time=0)
x_input = x_input.data

# original fit
tau = 1
model_org = LIM_class.LIM(tau)
model_org.fit(data_train.data, eps=eps)

eigenvalues_lst = []


# for t in range(1):
#
#     print("Iteration : {}".format(t))
#
#         # Forecast mean using LIM model
#     lim_integration, times_ = model_org.noise_integration(x_input, timesteps=20000, num_comp=len(pc_10), seed=t)
#     lim_integration = lim_integration.T
#
#     tau = 1
#     model = LIM_class.LIM(tau)
#     model.fit(lim_integration, eps=eps)
#
#     logar_matr = model.logarithmic_matrix
#     noise_matr = model.noise_covariance
#
#     # Plotting eigenvalues of G
#     eigenvalues, _, w = ut.matrix_decomposition(model.green_function)
#     eigenvalues_lst.append(eigenvalues)



lim_integration, times_ = model_org.noise_integration(x_input, timesteps=20000, num_comp=len(pc_10), seed=99)
lim_integration = lim_integration.T

tau = 1
modelxx = LIM_class.LIM(tau)
modelxx.fit(lim_integration, eps=eps)

lim_integration, times_ = model_org.noise_integration(x_input, timesteps=20000, num_comp=len(pc_10), seed=9999)
lim_integration = lim_integration.T

tau = 1
modelyy = LIM_class.LIM(tau)
modelyy.fit(lim_integration, eps=eps)

# lim_integration, times_ = model_org.noise_integration(x_input, timesteps=20000, num_comp=len(pc_10), seed=9999)
# lim_integration = lim_integration.T
#
# tau = 1
# modelzz = LIM_class.LIM(tau)
# modelzz.fit(lim_integration, eps=eps)
#
# lim_integration, times_ = model_org.noise_integration(x_input, timesteps=20000, num_comp=len(pc_10), seed=99999)
# lim_integration = lim_integration.T
#
# tau = 1
# modelcc = LIM_class.LIM(tau)
# modelcc.fit(lim_integration, eps=eps)


eigenvalues_org, _, w = ut.matrix_decomposition(model_org.green_function)
eigenvalues_xx, _, w = ut.matrix_decomposition(modelxx.green_function)
eigenvalues_yy, _, w = ut.matrix_decomposition(modelyy.green_function)
#eigenvalues_zz, _, w = ut.matrix_decomposition(modelzz.green_function)
#eigenvalues_cc, _, w = ut.matrix_decomposition(modelcc.green_function)



Covariance matrix has negative values!
0.3454143
t_delta: 0.9407243097032918
Negative eigenvalues detected.
Covariance matrix has negative values!
0.3454143
t_delta: 0.9407243097032918
We are in here


C:\Users\felix\PycharmProjects\ML_Climate_Science_Research_Project\LIM\LIM_class.py:260: RuntimeWarning: overflow encountered in multiply
  np.random.multivariate_normal([0 for n in range(num_comp)], self.noise_covariance))
C:\Users\felix\PycharmProjects\ML_Climate_Science_Research_Project\LIM\LIM_class.py:265: RuntimeWarning: invalid value encountered in add
  state_start = state_new
C:\Users\felix\PycharmProjects\ML_Climate_Science_Research_Project\LIM\LIM_class.py:260: RuntimeWarning: invalid value encountered in matmul
  np.random.multivariate_normal([0 for n in range(num_comp)], self.noise_covariance))
C:\Users\felix\PycharmProjects\ML_Climate_Science_Research_Project\LIM\LIM_class.py:80: RuntimeWarning: overflow encountered in matmul
  self.C_0 = (x @ x.T) / n_time
C:\Users\felix\PycharmProjects\ML_Climate_Science_Research_Project\LIM\LIM_class.py:80: RuntimeWarning: invalid value encountered in matmul
  self.C_0 = (x @ x.T) / n_time
C:\Users\felix\PycharmProjects\ML_Climate_Scie

LinAlgError: Array must not contain infs or NaNs

In [ ]:
x = np.arange(1, len(eigenvalues_org) + 1)

# Plot the eigenvalues
plt.figure(figsize=(15,10))
plt.plot(x, eigenvalues_org, marker='o', linestyle='-', color='r', markersize=4.5, markerfacecolor='r', linewidth=3,
         label='G_tau')
plt.plot(x, eigenvalues_xx, marker='o', linestyle='-', color='g', markersize=4, markerfacecolor='r', linewidth=1,
         label='G_new_2step')
plt.plot(x, eigenvalues_yy, marker='o', linestyle='-', color='orange', markersize=4, markerfacecolor='r', linewidth=1,
          label='G_new_1step')
# plt.plot(x, eigenvalues_zz, marker='o', linestyle='-', color='black', markersize=4, markerfacecolor='r', linewidth=1,
#          label='G_new_0.3delta')
# plt.plot(x, eigenvalues_cc, marker='o', linestyle='-', color='b', markersize=4, markerfacecolor='r', linewidth=1,
#          label='G_new_0.4delta')
# plt.plot(x, eigenvalues_lst[0], marker='o', linestyle='-', color='y', markersize=4, markerfacecolor='r', linewidth=1, label='G_new_0.5delta')

#for y in range(len(eigenvalues_lst)):
#    plt.plot(x, eigenvalues_lst[y], marker='o', linestyle='-', color='y', markersize=4, markerfacecolor='r', linewidth=2.5)
#    if y +1 == len(eigenvalues_lst):
#        plt.plot(x, eigenvalues_lst[y], marker='o', linestyle='-', color='y', markersize=4, markerfacecolor='r', linewidth=2.5, #label='G_new_tau1')





# Set the labels and title
plt.xlabel('Index', fontsize=20, labelpad=20)
plt.ylabel('Eigenvalue', fontsize=20, labelpad=20)
plt.title('Eigenvalues of G / G_new for different t_delta', fontsize=20)

# Set the x-axis tick labels
plt.xticks(x, fontsize=20)
plt.yticks(range(0,1), fontsize=20)
plt.legend(loc="lower left", fontsize=12)

# Display the plot
plt.savefig('./plots/svg/eigenvalues_G_combined_piControl_step.svg', format='svg')
plt.savefig('./plots/png/eigenvalues_G_combined_piControl_step.png', format='png')
plt.show()